# Binary Decision Tree Code

The code based submitted here builds a simple binary decision trees to take categorical variables as input and output a simple Decision Tree that classifies data into 2 [binary] classes e.g. BAD vs. Good, Yes vs. No

This work  will use the [LendingClub](https://www.lendingclub.com/) dataset to build a Decision Tree to discern between safe  versus Non-safe loans and predict in the future

## [LendingClub](https://www.lendingclub.com/) data set load & cleanup

Import csv data from local folder

In [18]:
import pandas as pd
loans = pd.read_csv('../lending-club-data.csv/lending-club-data.csv', \
                    low_memory=False)

In [19]:
loans['safe_loans'] = loans['bad_loans'].apply(lambda x : +1 if x==0 else -1)
loans.drop('bad_loans', axis =1)
print 'Loans:- (Rows, Columns)', loans.shape

Loans:- (Rows, Columns) (122607, 69)


In developing the code we use the following  4 categorical features to build and validate the Decision Tree code: 
1. Grade of the loan 
2. The length of the loan term
3. The home ownership status: own, mortgage, rent
4. Number of years of employment.
In the dataset, each of these features is a categorical feature. Since we are building a binary decision tree, we will have to convert this to binary data in a subsequent section using 1-hot encoding.

In [20]:
features = ['grade',              # grade of the loan
            'term',               # the term of the loan
            'home_ownership',     # home_ownership status: own, mortgage or rent
            'emp_length',         # number of years of employment
           ]
target = 'safe_loans'
loans = loans[features + [target]]

### Data set sampling to ensure class balance

A Balanced sample set with similar number of minority and majority class group is helpful for learning decision trees. While many different methodologies are available, we just choose to sample the majority class group to equate to the minority class

In [21]:
safe_loans_raw = loans[loans[target] == 1]
risky_loans_raw = loans[loans[target] == -1]

# Since there are less risky loans than safe loans, find the ratio of the sizes
# and use that percentage to undersample the safe loans.
percentage = len(risky_loans_raw)/float(len(safe_loans_raw))
safe_loans = safe_loans_raw.sample(len(risky_loans_raw), random_state = 1)
risky_loans = risky_loans_raw
loans_data = risky_loans.append(safe_loans)

print "Raw Proportions(minority/ majority class):", "{0:2.2f}%".format(percentage*100)
print "Percentage of safe loans                 :", len(safe_loans) / float(len(loans_data))
print "Percentage of risky loans                :", len(risky_loans) / float(len(loans_data))
print "Total number of loans in our new dataset :", len(loans_data)

Raw Proportions(minority/ majority class): 23.28%
Percentage of safe loans                 : 0.5
Percentage of risky loans                : 0.5
Total number of loans in our new dataset : 46300


### Transform categorical data into binary features

Since we are implementing **binary decision trees**, we transform our categorical data into binary data using 1-hot encoding. 
For instance, the **home_ownership** feature represents the home ownership status of the 
loanee, which is either `own`, `mortgage` or `rent`. For example, if a data point has the 
feature 
```
   {'home_ownership': 'RENT'}
```
we want to turn this into three features: 
```
 { 
   'home_ownership = OWN'      : 0, 
   'home_ownership = MORTGAGE' : 0, 
   'home_ownership = RENT'     : 1
 }
```

In [22]:
import numpy as np
loans_data = risky_loans.append(safe_loans)
#features = loans_data.columns
for feature in features:
    one_hot = pd.get_dummies(loans_data[feature], prefix = feature, prefix_sep='.')
    loans_data = loans_data.join(one_hot)
    loans_data.drop(feature, axis=1, inplace=True)

### Features in the Dataframe

In [23]:
features= loans_data.columns
features = features.drop('safe_loans')
print features
print "Number of features (after binarizing categorical variables) = %s" % len(features)

Index([u'grade.A', u'grade.B', u'grade.C', u'grade.D', u'grade.E', u'grade.F',
       u'grade.G', u'term. 36 months', u'term. 60 months',
       u'home_ownership.MORTGAGE', u'home_ownership.OTHER',
       u'home_ownership.OWN', u'home_ownership.RENT', u'emp_length.1 year',
       u'emp_length.10+ years', u'emp_length.2 years', u'emp_length.3 years',
       u'emp_length.4 years', u'emp_length.5 years', u'emp_length.6 years',
       u'emp_length.7 years', u'emp_length.8 years', u'emp_length.9 years',
       u'emp_length.< 1 year', u'emp_length.n/a'],
      dtype='object')
Number of features (after binarizing categorical variables) = 25


## Train-test split

We split the data into a train test split with 80% of the data in the training set and 20% of the data in the test set. We use `seed=1` so that everyone gets the same result.

In [24]:
def train_test_split(df, split = 0.8):
    np.random.seed(seed=1234)
    msk = np.random.rand(len(df)) < 0.8
    train = df[msk]
    test= df[~msk]
    return(train, test)

(train, test) = train_test_split(loans_data, 0.8)

print '# Rows in Loans Data',len(loans_data)
print '# Rows in train data',len(train)
print '# Rows in test data',len(test)

# Rows in Loans Data 46300
# Rows in train data 37030
# Rows in test data 9270


# Decision tree implementation

Here, binary decision trees is implemented from scratch. There are several steps involved in building a decision tree. The following are the high level steps:

## 1. Function to count number of Errors while predicting majority class

Prediction at an intermediate node works by predicting the **majority class** for all data points that belong to this node.

The function below calculates the number of **missclassified examples** when predicting the **majority class**. This will be used to help determine which feature is the best to split on at a given node of the tree.

**Note**: Keep in mind that in order to compute the number of errors for a majority classifier, we only need the label (y values) of the data points in the node. 

** Steps to follow **:
* ** Step 1:** Calculate the number of safe loans and risky loans.
* ** Step 2:** Since we are assuming majority class prediction, all the data points that are **not** in the majority class are considered **errors**.
* ** Step 3:** Return the number of **errors**.

In [25]:
def intermediate_node_num_errors(labels_in_node):
    # Corner case: If labels_in_node is empty, return 0
    if len(labels_in_node) == 0:
        return 0
    
    # Count the number of 1's (safe loans)
    safe_loans = (labels_in_node == 1).sum()
    # Count the number of -1's (risky loans)
    risky_loans = (labels_in_node == -1).sum()           
    # Return the number of errors that the majority classifier makes.
    if safe_loans > risky_loans:
        return(risky_loans)
    else:
        return(safe_loans)

#### Testing error count function

In [26]:
# Test case 1
example_labels = np.array([-1, -1, 1, 1, 1])
if intermediate_node_num_errors(example_labels) == 2:
    print 'Test passed!'
else:
    print 'Test 1 failed... try again!'

# Test case 2
example_labels =  np.array([-1, -1, 1, 1, 1, 1, 1])
if intermediate_node_num_errors(example_labels) == 2:
    print 'Test passed!'
else:
    print 'Test 2 failed... try again!'
    
# Test case 3
example_labels =  np.array([-1, -1, -1, -1, -1, 1, 1])
if intermediate_node_num_errors(example_labels) == 2:
    print 'Test passed!'
else:
    print 'Test 3 failed... try again!'

Test passed!
Test passed!
Test passed!


## 2. Function to pick best feature to split on

The function **best_splitting_feature** takes 3 arguments: 
1. The data (data frame which includes all of the feature columns and label column)
2. The features to consider for splits (a list of strings of column names to consider for splits)
3. The name of the target/label column (string)

The function will loop through the list of possible features, and consider splitting on each of them. It will calculate the classification error of each split and return the feature that had the smallest classification error when split on.

**classification error** is defined as follows:
$$
\mbox{classification error} = \frac{\mbox{# Errors}}{\mbox{# total examples}}
$$

Following are the steps: 
* **Step 1:** Loop over each feature in the feature list
* **Step 2:** Within the loop, the data is split into two groups: one group where all of the data has feature value 0 or False (we will call this the **left** split), and one group where all of the data has feature value 1 or True (we will call this the **right** split). The **left** split corresponds with 0 and the **right** split corresponds with 1 to ensure the implementation fits with the tree building process.
* **Step 3:** Calculate the number of misclassified examples in both groups of data and the above formula is used to compute the **classification error**.
* **Step 4:** If the computed error is smaller than the best error found so far, store this **feature and its error**.

Since we are only dealing with binary features, we do not have to consider thresholds for real-valued features.

In [28]:
def best_splitting_feature(data, features, target):
    
    best_feature = None # Keep track of the best feature 
    best_error = 10     # Keep track of the best error so far 
    # Note: Since error is always <= 1, we should intialize it with something larger than 1.

    # Convert to float to make sure error gets computed correctly.
    num_data_points = float(len(data))  
    
    # Loop through each feature to consider splitting on that feature
    for feature in features:
        
        # The left split will have all data points where the feature value is 0
        left_split = data[data[feature] == 0]
        
        # The right split will have all data points where the feature value is 1
        ## YOUR CODE HERE
        right_split = data[data[feature] == 1]
            
        # Calculate the number of misclassified examples in the left split.
        # Remember that we implemented a function for this! (It was called intermediate_node_num_errors)
        # YOUR CODE HERE
        left_errors = intermediate_node_num_errors(left_split[target])           

        # Calculate the number of misclassified examples in the right split.
        ## YOUR CODE HERE
        right_errors = intermediate_node_num_errors(right_split[target])
            
        # Compute the classification error of this split.
        # Error = (# of errors (left) + # of errors (right)) / (# of data points)
        ## YOUR CODE HERE
        error = float(left_errors+right_errors)/ float(num_data_points)
        #print 'feature + error', feature, error
        # If this is the best error we have found so far, store the feature as best_feature and the error as best_error
        ## YOUR CODE HERE
        if error < best_error:
            best_feature = feature
            best_error = error
    return best_feature # Return the best feature we found

#### Testing split function

In [29]:
best_splitting_feature(train, features, 'safe_loans')

'term. 36 months'

### 3. Function to Create Leaf and build the tree

Each node in the decision tree is represented as a dictionary which contains the following keys and possible values:

    { 
       'is_leaf'            : True/False.
       'prediction'         : Prediction at the leaf node.
       'left'               : (dictionary corresponding to the left tree).
       'right'              : (dictionary corresponding to the right tree).
       'splitting_feature'  : The feature that this node splits on.
    }


The function below creates a leaf node given a set of target values. 

In [30]:
def create_leaf(target_values):
    
    # Create a leaf node
    leaf = {'is_leaf': True,
            'prediction': None,
            'splitting_feature' : None,
            'left' : None,
            'right' : None}   ## YOUR CODE HERE
    
    # Count the number of data points that are +1 and -1 in this node.
    num_ones = len(target_values[target_values == +1])
    num_minus_ones = len(target_values[target_values == -1])
    
    # For the leaf node, set the prediction to be the majority class.
    # Store the predicted class (1 or -1) in leaf['prediction']
    if num_ones > num_minus_ones:
        leaf['prediction'] =    1      ## YOUR CODE HERE
    else:
        leaf['prediction'] =   -1     ## YOUR CODE HERE
        
    # Return the leaf node        
    return leaf 

## 4. Function to build the tree in a top down manner

This function learns the decision tree recursively and implements 3 stopping conditions:

**Stopping condition 1:** All data points in a node are from the same class.

**Stopping condition 2:** No more features to split on.

**Stopping condition 3:** Stopping condition based on the **max_depth** of the tree. By not letting the tree grow too deep, we will save computational effort in the learning process. 

**Stopping condition 4:** minimum node size:
* **Step 1:** The function **reached_minimum_node_size** detects whether we have hit the base case, i.e., the node does not have enough data points and should be turned into a leaf. 
* **Step 2:** Return a leaf using create_leaf

**Stopping condition 5: minimum error reduction:**
This has to come after finding the best splitting feature so we can calculate the error after splitting in order to calculate the error reduction.
* **Step 1:** Calculate the **classification error before splitting**.  
* **Step 2:** Calculate the **classification error after splitting**. This requires calculating the number of errors in the left and right splits, and then dividing by the total number of examples.
* **Step 3:** The function **error_reduction** detects whether  the reduction in error is less than the constant provided (`min_error_reduction`). 
* **Step 4:** Return a leaf using create_leaf

In [31]:
def error_reduction(error_before_split, error_after_split):
    # Return the error before the split minus the error after the split.
    gain = (error_before_split - error_after_split)
    return(gain)

In [32]:
def reached_minimum_node_size(data, min_node_size):
    # Return True if the number of data points is less than or equal to the minimum node size.
    return( True if len(data) <= min_node_size else False)

In [33]:
def count_nodes(tree):
    if tree['is_leaf']:
        return 1
    return 1 + count_nodes(tree['left']) + count_nodes(tree['right'])

In [34]:
def decision_tree_create(data, features, target, current_depth = 0, 
                         max_depth = 10, min_node_size=1, 
                         min_error_reduction=0.0):
    
    remaining_features = features[:] # Make a copy of the features.
    
    target_values = data[target]
    print "--------------------------------------------------------------------"
    print "Subtree, depth = %s (%s data points)." % (current_depth, len(target_values))
    
    
    # Stopping condition 1: All nodes are of the same type.
    if intermediate_node_num_errors(target_values) == 0:
        print "Stopping condition 1 reached. All data points have the same class."                
        return create_leaf(target_values)
    
    # Stopping condition 2: No more features to split on.
    if remaining_features == []:
        print "Stopping condition 2 reached. No remaining features."                
        return create_leaf(target_values)    
    
    # Early stopping condition 1: Reached max depth limit.
    if current_depth >= max_depth:
        print "Stopping condition 3 reached. Reached maximum depth."
        return create_leaf(target_values)
    
    # Early stopping condition 2: Reached the minimum node size.
    # If the number of data points is less than or equal to the minimum size, return a leaf.
    if  reached_minimum_node_size(data, min_node_size) == True:
        print "Stopping condition 4 reached. Reached minimum node size."
        return  create_leaf(target_values)
    
    # Find the best splitting feature
    splitting_feature = best_splitting_feature(data, features, target)
    
    # Split on the best feature that we found. 
    left_split = data[data[splitting_feature] == 0]
    right_split = data[data[splitting_feature] == 1]
    
    # Early stopping condition 3: Minimum error reduction
    # Calculate the error before splitting (number of misclassified examples 
    # divided by the total number of examples)
    error_before_split = intermediate_node_num_errors(target_values) / float(len(data))
    #print error_before_split
    # Calculate the error after splitting (number of misclassified examples 
    # in both groups divided by the total number of examples)
    left_errors =    intermediate_node_num_errors(left_split[target]) 
    right_errors =   intermediate_node_num_errors(right_split[target])
    error_after_split = (left_errors + right_errors) / float(len(data))
    #print error_before_split
    # If the error reduction is LESS THAN OR EQUAL TO min_error_reduction, return a leaf.
    if error_reduction(error_before_split, error_after_split) <= min_error_reduction:
        print "Stopping condition 5 reached. Minimum error reduction.",\
                error_reduction(error_before_split, error_after_split)
        return create_leaf(target_values)
      
    remaining_features.drop(splitting_feature)
    print "Split on feature %s. (%s, %s)" % (\
                      splitting_feature, len(left_split), len(right_split))
    
        # Repeat (recurse) on left and right subtrees
    left_tree = decision_tree_create(left_split, remaining_features, target, 
                                     current_depth + 1, max_depth, min_node_size, min_error_reduction)        
    
    
    right_tree = decision_tree_create(right_split, remaining_features, target, 
                                     current_depth + 1, max_depth, min_node_size, min_error_reduction)        
    
    return {'is_leaf'          : False, 
            'prediction'       : None,
            'splitting_feature': splitting_feature,
            'left'             : left_tree, 
            'right'            : right_tree}

In [35]:
small_decision_tree = decision_tree_create(train, features, 'safe_loans', max_depth = 2, 
                                        min_node_size = 10, min_error_reduction=0.0)
if count_nodes(small_decision_tree) == 7:
    print 'Test passed!'
else:
    print 'Test failed... try again!'
    print 'Number of nodes found                :', count_nodes(small_decision_tree)
    print 'Number of nodes that should be there : 7' 

--------------------------------------------------------------------
Subtree, depth = 0 (37030 data points).
Split on feature term. 36 months. (9261, 27769)
--------------------------------------------------------------------
Subtree, depth = 1 (9261 data points).
Split on feature grade.A. (9146, 115)
--------------------------------------------------------------------
Subtree, depth = 2 (9146 data points).
Stopping condition 3 reached. Reached maximum depth.
--------------------------------------------------------------------
Subtree, depth = 2 (115 data points).
Stopping condition 3 reached. Reached maximum depth.
--------------------------------------------------------------------
Subtree, depth = 1 (27769 data points).
Split on feature grade.D. (23067, 4702)
--------------------------------------------------------------------
Subtree, depth = 2 (23067 data points).
Stopping condition 3 reached. Reached maximum depth.
-----------------------------------------------------------------

## Prediction Function

We make predictions from the decision tree with a simple recursive function. Below, the function classify is called, which takes in a learned tree and point x to classify. We include an option annotate that describes the prediction path when set to True.

In [36]:
def classify(tree, x, annotate = False):   
    # if the node is a leaf node.
    if tree['is_leaf']:
        if annotate: 
            print "At leaf, predicting %s" % tree['prediction']
        return tree['prediction'] 
    else:
        # split on feature.
        split_feature_value = x[tree['splitting_feature']]
        if annotate: 
            print "Split on %s = %s" % (tree['splitting_feature'], split_feature_value)
        if split_feature_value == 0:
            return classify(tree['left'], x, annotate)
        else:
            return classify(tree['right'], x, annotate)

In [37]:
predict = []
for i in xrange(0,len(train)):
    predict.append(classify(small_decision_tree,train.iloc[i]))
print 'Training Error:', float(sum(predict != train['safe_loans']))/float(len(train))

Training Error: 0.402862543883


## Classification Error
The end goal is to evaluate classification error of the tree. 
**Classification error** is defined as follows:
$$
\mbox{classification error} = \frac{\mbox{# errors}}{\mbox{# total examples}}
$$

`evaluate_classification_error` takes following input:
1. `tree` (as described above)
2. `data` (an SFrame)
3. `target` (a string - the name of the target/label column)

This function calculates a prediction (class label) for each row in `data` using the decision `tree` and return the classification error computed using the above formula.

In [38]:
def evaluate_classification_error(tree, data, target):
    # Apply the classify(tree, x) to each row in your data
    predict = []
    for i in xrange(0,len(data)):
        predict.append(classify(tree,data.iloc[i]))
    # Once you've made the predictions, calculate the classification error and return it
    err = data[predict!=data['safe_loans']]
    classification_error = float(len(err))/float(len(data))
    return (classification_error)

In [39]:
# Train Error:
print "Training Classification Error:",evaluate_classification_error(small_decision_tree, train, 'safe_loans')
# Test Error:
print "Test Classification Error:",evaluate_classification_error(small_decision_tree, test, 'safe_loans')

Training Classification Error: 0.402862543883
Test Classification Error: 0.40021574973


## Utility to visualizse the decision tree

The following is a recursive print function that prints out the decision tree with following arguments:
* tree: Name of Decision Tree
* max_d: Maximum Depth specified for printing
* d    : 

Note: Works for the linked dictionary tree structure provided here

In [40]:
def print_decision_tree(tree, max_d = None, d = 0, pre = ''):
    if tree['is_leaf']:
        print(pre + '* Predict %s' % tree['prediction'])
    elif max_d is not None and d >= max_d:
        print(pre + '...')
    else:
        print(pre + '+ %s = 0' % tree['splitting_feature'])
        print_decision_tree(tree['left'], max_d, d + 1, pre + '|   ')
        print(pre + 'L %s = 1' % tree['splitting_feature'])
        print_decision_tree(tree['right'], max_d, d + 1, pre + '    ')

In [43]:
print_decision_tree(small_decision_tree, max_d = 6, d = 0, pre = '')

+ term. 36 months = 0
|   + grade.A = 0
|   |   * Predict -1
|   L grade.A = 1
|       * Predict 1
L term. 36 months = 1
    + grade.D = 0
    |   * Predict 1
    L grade.D = 1
        * Predict -1


In [60]:
print_decision_tree(small_decision_tree, max_d = 6, d = 5, pre = '')

+ term. 36 months = 0
|   ...
L term. 36 months = 1
    ...


In [61]:
print_decision_tree(small_decision_tree, max_d = 6, d = 4, pre = '|')

|+ term. 36 months = 0
||   + grade.A = 0
||   |   * Predict -1
||   L grade.A = 1
||       * Predict 1
|L term. 36 months = 1
|    + grade.D = 0
|    |   * Predict 1
|    L grade.D = 1
|        * Predict -1


# Postscript

This is a simple Decision Tree implementation to solve Binary Classification problem. The complexity of the tree built can be controlled by the depth and number of leaves, to manage overfitting while giving better accuracy. In the current tree model the performance is slightly better than a random split!